In [1]:
import os

from utils.auxiliary_func import *
from utils.plots_func import *
from flim_analysis.spatial_analysis.spatial_func import *
import config.const as const

## Define Parameters

In [2]:
patch_size = 1500
overlap = 0.75
fold_method='stratified_kfold'

In [3]:
DATA_PREPARATION_DIR = const.DATA_PREPARATION_DIR

## Check correlation between density and patch probability 

#### Note on Data Preparation

The csv data used in this code was generated in the following notebook:

**`TNBC_FLIM/notebooks/analysis_paper_result_reproduce/Supplementary/SDistributionNumNucluiPerPatch_preparation.ipynb`**

Please make sure to run that notebook first to prepare the necessary data files before executing this one.

### Create data density and patch probability

In [4]:
patch_df = read_patches_files(patch_size, overlap)

In [5]:
hist_base_dir = os.path.join(const.DISTRIBUTION_RESULTS_PATCH_DIR, f"overlap_{patch_size}", fold_method)
seed_num = find_seed_best_auc(hist_base_dir)

mean_auc :0.6569855072463768
best_auc: 0.7753623188405797
median_auc: 0.6572463768115941
Seed corresponding to best AUC: 3006


In [6]:
best_hist_df_1500 = extract_hist_results(hist_base_dir, seed_num)

In [11]:
# Define the path
nuclei_per_patch_path = os.path.join(DATA_PREPARATION_DIR, f'nuclei_count_per_patch_{patch_size}_df.csv')

# Read the file, enforcing string types for the specified columns
nuclei_per_patch_df = pd.read_csv(nuclei_per_patch_path, dtype={'leap_ID': str, 'patch_ID': str})
nuclei_per_patch_df.head()

,patch_ID,leap_ID,nuclei_count
0,0_0_1500_1500,028,6536
1,0_1005_1500_2505,117,8000
2,0_10199_1500_11699,128,27165
3,0_10228_1500_11728,034,61
4,0_103_1500_1603,034,11949


In [12]:
# Merge on leap_ID and patch_ID
merged_df = pd.merge(best_hist_df_1500, nuclei_per_patch_df, on=['leap_ID', 'patch_ID'], how='left')
merged_df.head()

,patient_id,leap_ID,y_true,prob_results,patch_ID,nuclei_count
0,11,015,0,0.482645,1128_1127_2628_2627,548
1,11,015,0,0.517836,1128_1502_2628_3002,410
2,11,015,0,0.466807,1128_1877_2628_3377,310
3,11,015,0,0.364429,1128_2252_2628_3752,118
4,11,015,0,0.545767,1128_2_2628_1502,5172


In [13]:
output_nuclei_count_patch_prob_path = os.path.join(DATA_PREPARATION_DIR, f'nuclei_count_and_patch_probability_{patch_size}_df.csv')
merged_df.to_csv(output_nuclei_count_patch_prob_path, index=False)